In [ ]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils, callbacks
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

from tensorflow.keras.utils import custom_object_scope
from matplotlib.colors import ListedColormap
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
import zipfile
from PIL import Image
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LambdaCallback
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/
! mkdir Music_Symbols
%cd Music_Symbols

!cp /content/drive/MyDrive/MESTRADO_UFU/bioinspirada/trabalho_automato_2/Music_Symbols.zip .

!unzip Music_Symbols.zip
!rm Music_Symbols.zip

## Data

In [ ]:
import os
import zipfile
import numpy as np
from PIL import Image
import tensorflow as tf


def load_and_preprocess_clef(zip_file_path):
    """
    Load and preprocess images from clef folders inside the given zip file.

    :param zip_file_path: Path to the zip file containing the Music_Symbols dataset.
    :return: Preprocessed clef images and their one-hot encoded labels.
    """
    return _load_and_preprocess_from_zip(zip_file_path, prefix="CLEF", target_size=(32, 32))


def load_and_preprocess_accidental(zip_file_path):
    """
    Load and preprocess images from accidental folders inside the given zip file.

    :param zip_file_path: Path to the zip file containing the Music_Symbols dataset.
    :return: Preprocessed accidental images and their one-hot encoded labels.
    """
    return _load_and_preprocess_from_zip(zip_file_path, prefix="ACCIDENTAL", target_size=(32, 32))


def _load_and_preprocess_from_zip(zip_file_path, prefix, target_size=(32, 32)):
    """
    Generic function to load and preprocess images from specified folders in the zip file.

    :param zip_file_path: Path to the zip file containing the dataset.
    :param prefix: Prefix of folders to process (e.g., "CLEF" or "ACCIDENTAL").
    :param target_size: Tuple indicating the target size to resize images (default is 32x32).
    :return: Preprocessed images and their one-hot encoded labels.
    """
    temp_dir = "temp_extracted"
    os.makedirs(temp_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    images = []
    labels = []
    folder_names = [f for f in os.listdir(temp_dir) if f.startswith(prefix)]
    for folder_name in folder_names:
        label = folder_name.split("_")[-1]
        folder_path = os.path.join(temp_dir, folder_name)

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            try:

                image = Image.open(file_path).convert("RGB")
                image = image.resize(target_size)
                image = np.array(image) / 255.0

                images.append(image)
                labels.append(label)

            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    for root, dirs, files in os.walk(temp_dir, topdown=False):
        for file in files:
            os.remove(os.path.join(root, file))
        for dir in dirs:
            os.rmdir(os.path.join(root, dir))
    os.rmdir(temp_dir)


    unique_labels = sorted(set(labels))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    labels = [label_to_index[label] for label in labels]


    images = np.array(images)
    labels = np.array(labels)

    labels = tf.keras.utils.to_categorical(labels, len(unique_labels))

    return images, labels

def load_and_preprocess_mnist():
    """
    Load and preprocess the MNIST dataset for VGG16.

    :return: Preprocessed MNIST images and their one-hot encoded labels.
    """

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


    x_data = np.concatenate([x_train, x_test], axis=0)
    y_data = np.concatenate([y_train, y_test], axis=0)

    x_data = np.stack((x_data,) * 3, axis=-1)

    x_data_resized = np.array([np.array(Image.fromarray(img).resize((32, 32))) for img in x_data])

    x_data_resized = x_data_resized / 255.0

    y_data = tf.keras.utils.to_categorical(y_data, 10)

    return x_data_resized, y_data

zip_file_path = "/content/drive/MyDrive/MESTRADO_UFU/bioinspirada/trabalho_automato_2/Music_Symbols.zip"


## VGG Convolutional Layers

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
def create_vgg16(input_shape=(32, 32, 1), num_classes=1000):
    model = models.Sequential()

    model.add(layers.Conv2D(64, 3, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Conv2D(128, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(128, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256, activation='relu'))

    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


In [ ]:
model = create_vgg16((32, 32, 1), 10)
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 32, 32, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 1, 1, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 14,913,226 (56.89 MB)

 Trainable params: 14,913,226 (56.89 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def cross_validate_lenet5(dados):
    if dados == 'clef':
      x_data, y_data = load_and_preprocess_clef(zip_file_path)
    elif dados == 'accidentals':
      x_data, y_data = load_and_preprocess_accidental(zip_file_path)
    elif dados == 'mnist':
      x_data, y_data = load_and_preprocess_mnist()

    n_classes = y_data.shape[1]
    input_shape = x_data.shape[1:]

    kf = KFold(n_splits=4, shuffle=True, random_state=42)

    save_epochs = [5, 15, 25, 50, 100]
    #save_epochs = [5]
    metrics = {epoch: [] for epoch in save_epochs}
    evaluation_times = {epoch: [] for epoch in save_epochs}
    training_times = {epoch: [] for epoch in save_epochs}

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_data)):
        print(f"Starting fold {fold + 1}")

        x_train_full, x_val = x_data[train_idx], x_data[val_idx]
        y_train_full, y_val = y_data[train_idx], y_data[val_idx]

        x_train, x_val, y_train, y_val = train_test_split(
            x_train_full, y_train_full, test_size=0.2, random_state=42
        )

        model = create_vgg16(input_shape, n_classes)
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
        model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

        checkpoint_path = f"lenet5_fold{fold + 1}_epoch_{{epoch:02d}}.h5"

        epoch_start_time = None
        current_epoch = 0
        training_times_accumulator = 0

        def save_on_epoch(epoch, logs):
            nonlocal epoch_start_time, training_times_accumulator, current_epoch

            if epoch_start_time is not None:
                training_times_accumulator += time.time() - epoch_start_time

            epoch_start_time = time.time()
            current_epoch = epoch + 1

            if current_epoch in save_epochs:
                training_times[current_epoch].append(training_times_accumulator)
                model.save(checkpoint_path.format(epoch=current_epoch))

        checkpoint_callback = callbacks.LambdaCallback(on_epoch_end=save_on_epoch)


        model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=max(save_epochs),
            batch_size=32,
            callbacks=[checkpoint_callback],
            verbose=1
        )

        for epoch in save_epochs:
            model_path = f"lenet5_fold{fold + 1}_epoch_{epoch:02d}.h5"
            saved_model = models.load_model(model_path)

            start_time = time.time()
            val_loss, val_accuracy = saved_model.evaluate(x_val, y_val, verbose=0)
            end_time = time.time()

            elapsed_time = end_time - start_time
            evaluation_times[epoch].append(elapsed_time)

            metrics[epoch].append(val_accuracy)
            print(f"Fold {fold + 1}, Epoch {epoch}: Val Accuracy: {val_accuracy:.4f}, Evaluation Time: {elapsed_time:.2f} seconds")

    for epoch in save_epochs:
        print('BASE: ', dados)
        mean_acc = np.mean(metrics[epoch])
        std_acc = np.std(metrics[epoch])
        mean_eval_time = np.mean(evaluation_times[epoch])
        mean_train_time = np.mean(training_times[epoch]) if training_times[epoch] else 0.0
        print(f"Epoch {epoch}: Mean Accuracy: {mean_acc:.4f}, Std Dev: {std_acc:.4f}, Mean Eval Time: {mean_eval_time:.2f} seconds, Mean Train Time: {mean_train_time:.2f} seconds")

dados = ['accidentals', 'clef', 'mnist']

for x in dados:
  cross_validate_lenet5(x)




Starting fold 1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 266ms/step - accuracy: 0.2842 - loss: 1.3773 - val_accuracy: 0.5831 - val_loss: 0.9042
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6713 - loss: 0.8039 - val_accuracy: 0.8610 - val_loss: 0.3395
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.9155 - loss: 0.2668 - val_accuracy: 0.9729 - val_loss: 0.0962
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9658 - loss: 0.1183 - val_accuracy: 0.9831 - val_loss: 0.0705
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9802 - loss: 0.0837

37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.9802 - loss: 0.0831 - val_accuracy: 0.9898 - val_loss: 0.0200
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9953 - loss: 0.0132 - val_accuracy: 0.9966 - val_loss: 0.0126
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9909 - loss: 0.0330 - val_accuracy: 0.9966 - val_loss: 0.0216
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.9969 - loss: 0.0116 - val_accuracy: 0.9831 - val_loss: 0.0416
Epoch 9/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9912 - loss: 0.0479 - val_accuracy: 0.9966 - val_loss: 0.0116
Epoch 10/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9988 - loss: 0.0042 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 11/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.9997 - loss: 0.0035 - val_accuracy: 0.9966 - val_loss: 0.0081
Epoch 12/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - va

37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 1.0000 - loss: 1.0860e-04 - val_accuracy: 1.0000 - val_loss: 2.6430e-04
Epoch 16/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 6.8178e-05 - val_accuracy: 1.0000 - val_loss: 1.1899e-04
Epoch 17/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 1.7547e-05 - val_accuracy: 1.0000 - val_loss: 6.0269e-04
Epoch 18/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 1.3675e-05 - val_accuracy: 1.0000 - val_loss: 3.0303e-04
Epoch 19/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 7.1539e-06 - val_accuracy: 1.0000 - val_loss: 1.4096e-04
Epoch 20/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 6.3722e-06 - val_accuracy: 1.0000 - val_loss: 1.0042e-04
Epoch 21/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 5.9679e-06 - val_accuracy: 1.0000 - val_loss: 2.9837e-04
Epoch 22/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step -

...

Fold 4, Epoch 5: Val Accuracy: 0.9894, Evaluation Time: 3.75 seconds


Fold 4, Epoch 15: Val Accuracy: 0.9910, Evaluation Time: 3.75 seconds


Fold 4, Epoch 25: Val Accuracy: 0.9922, Evaluation Time: 3.74 seconds


Fold 4, Epoch 50: Val Accuracy: 0.9899, Evaluation Time: 3.71 seconds


Fold 4, Epoch 100: Val Accuracy: 0.9916, Evaluation Time: 3.70 seconds
BASE:  mnist
Epoch 5: Mean Accuracy: 0.9874, Std Dev: 0.0013, Mean Eval Time: 3.79 seconds, Mean Train Time: 183.10 seconds
BASE:  mnist
Epoch 15: Mean Accuracy: 0.9907, Std Dev: 0.0017, Mean Eval Time: 3.97 seconds, Mean Train Time: 640.02 seconds
BASE:  mnist
Epoch 25: Mean Accuracy: 0.9922, Std Dev: 0.0002, Mean Eval Time: 3.76 seconds, Mean Train Time: 1096.44 seconds
BASE:  mnist
Epoch 50: Mean Accuracy: 0.9911, Std Dev: 0.0016, Mean Eval Time: 3.78 seconds, Mean Train Time: 2233.56 seconds
BASE:  mnist
Epoch 100: Mean Accuracy: 0.9932, Std Dev: 0.0009, Mean Eval Time: 3.75 seconds, Mean Train Time: 4495.03 seconds


## CA-NN

In [ ]:
def load_and_preprocess_clef(zip_file_path):
    """
    Load and preprocess images from clef folders inside the given zip file.

    :param zip_file_path: Path to the zip file containing the Music_Symbols dataset.
    :return: Binarized clef images and their one-hot encoded labels.
    """
    return _load_and_preprocess_from_zip(zip_file_path, prefix="CLEF")


def load_and_preprocess_accidental(zip_file_path):
    """
    Load and preprocess images from accidental folders inside the given zip file.

    :param zip_file_path: Path to the zip file containing the Music_Symbols dataset.
    :return: Binarized accidental images and their one-hot encoded labels.
    """
    return _load_and_preprocess_from_zip(zip_file_path, prefix="ACCIDENTAL")


def _load_and_preprocess_from_zip(zip_file_path, prefix, target_size=(32, 32)):
    """
    Generic function to load and preprocess images from specified folders in the zip file.

    :param zip_file_path: Path to the zip file containing the dataset.
    :param prefix: Prefix of folders to process (e.g., "CLEF" or "ACCIDENTAL").
    :param target_size: Tuple indicating the target size to resize images (default is 32x32).
    :return: Binarized images and their one-hot encoded labels.
    """

    temp_dir = "temp_extracted"
    os.makedirs(temp_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    images = []
    labels = []

    folder_names = [f for f in os.listdir(temp_dir) if f.startswith(prefix)]
    for folder_name in folder_names:
        label = folder_name.split("_")[-1]
        folder_path = os.path.join(temp_dir, folder_name)

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            image = Image.open(file_path).convert("L")
            image = image.resize(target_size)
            image = np.array(image)
            image = (image >= 128).astype(np.float32)
            images.append(image)
            labels.append(label)

    for root, dirs, files in os.walk(temp_dir, topdown=False):
        for file in files:
            os.remove(os.path.join(root, file))
        for dir in dirs:
            os.rmdir(os.path.join(root, dir))
    os.rmdir(temp_dir)


    unique_labels = sorted(set(labels))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    labels = [label_to_index[label] for label in labels]

    images = np.array(images)
    labels = np.array(labels)

    images = np.expand_dims(images, axis=-1)

    labels = tf.keras.utils.to_categorical(labels, len(unique_labels))

    return images, labels

def load_and_preprocess_mnist():
    """
    Load and preprocess the MNIST dataset with binarization.

    :return: Binarized MNIST images and their one-hot encoded labels.
    """

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    x_data = np.concatenate([x_train, x_test], axis=0)
    y_data = np.concatenate([y_train, y_test], axis=0)

    x_data = (x_data >= 128).astype(np.float32)

    x_data = np.expand_dims(x_data, axis=-1)

    y_data = tf.keras.utils.to_categorical(y_data, 10)

    return x_data, y_data

zip_file_path = "/content/drive/MyDrive/MESTRADO_UFU/bioinspirada/trabalho_automato_2/Music_Symbols.zip"

In [ ]:
class CALayer(tf.keras.layers.Layer):
    def __init__(self, kernel_size=(5, 5), **kwargs):
        super(CALayer, self).__init__(**kwargs)
        self.kernel_size = kernel_size
        self.total_neighbors = kernel_size[0] * kernel_size[1] - 1  # Exclude the center cell

    def build(self, input_shape):
        # Dynamically create the kernel based on the input shape's channels
        input_channels = input_shape[-1]
        self.kernel = tf.ones((self.kernel_size[0], self.kernel_size[1], input_channels, input_channels),
                              dtype=tf.float32)

    def call(self, inputs):

        padding_size = self.kernel_size[0] // 2
        inputs_padded = tf.pad(inputs, [[0, 0], [padding_size, padding_size], [padding_size, padding_size], [0, 0]], mode="CONSTANT")

        neighbors = tf.nn.convolution(inputs_padded, self.kernel, padding="VALID") - inputs

        ngol_output = self.apply_ngol(inputs, neighbors)
        nhighlife_output = self.apply_nhighlife(inputs, neighbors)
        nbb_output = self.apply_nbb(inputs, neighbors)

        return tf.concat([ngol_output, nhighlife_output, nbb_output], axis=-1)

    def apply_nhighlife(self, inputs, neighbors):

        inputs = tf.cast(inputs > 0.000001, tf.float32)
        neighbors = neighbors - inputs

        # Proportion thresholds
        required_neighbors_2 = (2 / 8) * self.total_neighbors
        required_neighbors_4 = (4 / 8) * self.total_neighbors
        required_neighbors_3 = (3 / 8) * self.total_neighbors
        required_neighbors_6 = (6 / 8) * self.total_neighbors

        # Current cell states
        is_alive = tf.cast(inputs == 1, tf.float32)
        is_dead = tf.cast(inputs == 0, tf.float32)


        survive = tf.cast((neighbors >= required_neighbors_2) & (neighbors <= required_neighbors_3), tf.float32) * is_alive

        reproduce = tf.cast((neighbors == required_neighbors_3) | (neighbors == required_neighbors_6), tf.float32) * is_dead

        return survive + reproduce

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (input_shape[-1] * 3,)

    def apply_ngol(self, inputs, neighbors):

        inputs = tf.cast(inputs > 0.01, tf.float32)
        neighbors = neighbors - inputs

        # Proportion thresholds
        required_neighbors_2 = (2 / 8) * self.total_neighbors
        required_neighbors_4 = (4 / 8) * self.total_neighbors
        required_neighbors_3 = (3 / 8) * self.total_neighbors

        # Current cell states
        is_alive = tf.cast(inputs == 1, tf.float32)
        is_dead = tf.cast(inputs == 0, tf.float32)

        survive = tf.cast((neighbors >= required_neighbors_2) & (neighbors <= required_neighbors_3), tf.float32) * is_alive

        reproduce = tf.cast(neighbors == required_neighbors_3, tf.float32) * is_dead

        return survive + reproduce



    def apply_nbb(self, inputs, neighbors, b=3):

        neighbors = neighbors - tf.cast(inputs == 1, tf.float32)
        required_neighbor = (b / 8) * self.total_neighbors

        alive = tf.cast(inputs == 1, tf.float32)  # Alive state (White)
        dying = tf.cast(inputs == 2, tf.float32)  # Dying state (Gray)
        dead = tf.cast(inputs == 0, tf.float32)   # Dead state (Black)

        alive_to_dying = alive
        dying_to_dead = dying
        dead_to_alive = tf.cast(
            neighbors == required_neighbor,
            tf.float32
        ) * dead

        return dead_to_alive * 1 + alive_to_dying * 2 + dying_to_dead * 0

@tf.keras.utils.register_keras_serializable()
def threshold_to_binary(tensor):
    """
    Convert a tensor to binary values where any value > 1e-6 becomes 1.
    """
    return tf.cast(tensor > 1e-6, tf.float32)


def create_vgg16_with_ca(input_shape=(224, 224, 3), num_classes=1000):
    """
    Full VGG-16 implementation using CALayer and variables for better modularity.

    :param input_shape: Shape of the input image (default is 224x224x3 for ImageNet).
    :param num_classes: Number of output classes (default is 1000 for ImageNet).
    :return: A Keras Model instance.
    """
    inputs = tf.keras.Input(shape=input_shape, name="input")
    x = inputs

    x = CALayer(kernel_size=(3, 3), name="ca_layer_1")(x)
    x = layers.Activation("relu", name="activation_ca_1")(x)
    x = tf.keras.layers.Lambda(threshold_to_binary, name="binary_ca_layer_1")(x)  # Threshold to binary
    x = CALayer(kernel_size=(3, 3), name="ca_layer_2")(x)
    x = layers.Activation("relu", name="activation_ca_2")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="max_pool_1")(x)

    x = tf.keras.layers.Lambda(threshold_to_binary, name="binary_ca_layer_2")(x)  # Threshold to binary
    x = CALayer(kernel_size=(3, 3), name="ca_layer_3")(x)
    x = layers.Activation("relu", name="activation_ca_3")(x)
    x = tf.keras.layers.Lambda(threshold_to_binary, name="binary_ca_layer_3")(x)  # Threshold to binary
    x = CALayer(kernel_size=(3, 3), name="ca_layer_4")(x)
    x = layers.Activation("relu", name="activation_ca_4")(x)

    x = tf.keras.layers.Lambda(threshold_to_binary, name="binary_ca_layer_4")(x)  # Threshold to binary
    x = CALayer(kernel_size=(3, 3), name="ca_layer_5")(x)
    x = layers.Activation("relu", name="activation_ca_5")(x)
    x = tf.keras.layers.Lambda(threshold_to_binary, name="binary_ca_layer_5")(x)  # Threshold to binary
    x = CALayer(kernel_size=(3, 3), name="ca_layer_6")(x)
    x = layers.Activation("relu", name="activation_ca_6")(x)

    x = layers.Flatten(name="flatten")(x)
    x = layers.Dense(256, activation="relu", name="dense_1")(x)
    x = layers.Dense(256, activation="relu", name="dense_2")(x)
    x = layers.Dropout(0.5, name="dropout_2")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="output")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="VGG16_with_CA")

    return model


def create_vgg_with_trainable_dense(input_shape, n_classes):
    model = create_vgg16_with_ca(input_shape, n_classes)
    for layer in model.layers:
        if isinstance(layer, CALayer):
            layer.trainable = False  # Fix CA layers
    return model

In [ ]:
model = create_vgg_with_trainable_dense((32, 32, 1), 10)
model.summary()

Model: "VGG16_with_CA"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 32, 32, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ ca_layer_1 (CALayer)                 │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_ca_1 (Activation)         │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ binary_ca_layer_1 (Lambda)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ ca_layer_2 (CALayer)                 │ (None, 32, 32, 9)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_ca_2 (Activation)         │ (None, 32, 32, 9)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pool_1 (MaxPooling2D)            │ (None, 16, 16, 9)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ binary_ca_layer_2 (Lambda)           │ (None, 16, 16, 9)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ ca_layer_3 (CALayer)                 │ (None, 16, 16, 27)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_ca_3 (Activation)         │ (None, 16, 16, 27)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ binary_ca_layer_3 (Lambda)           │ (None, 16, 16, 27)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ ca_layer_4 (CALayer)                 │ (None, 16, 16, 81)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_ca_4 (Activation)         │ (None, 16, 16, 81)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ binary_ca_layer_4 (Lambda)           │ (None, 16, 16, 81)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ ca_layer_5 (CALayer)                 │ (None, 16, 16, 243)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_ca_5 (Activation)         │ (None, 16, 16, 243)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ binary_ca_layer_5 (Lambda)           │ (None, 16, 16, 243)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ ca_layer_6 (CALayer)                 │ (None, 16, 16, 729)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_ca_6 (Activation)         │ (None, 16, 16, 729)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │      47,776,000 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 47,844,362 (182.51 MB)

 Trainable params: 47,844,362 (182.51 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def cross_validate_lenet5(dados):
    if dados == 'clef':
        x_data, y_data = load_and_preprocess_clef(zip_file_path)
    elif dados == 'accidentals':
        x_data, y_data = load_and_preprocess_accidental(zip_file_path)
    elif dados == 'mnist':
        x_data, y_data = load_and_preprocess_mnist()

    n_classes = y_data.shape[1]
    input_shape = x_data.shape[1:]

    kf = KFold(n_splits=4, shuffle=True, random_state=42)

    save_epochs = [5, 15, 25, 50, 100]
    #save_epochs = [5]
    metrics = {epoch: [] for epoch in save_epochs}
    evaluation_times = {epoch: [] for epoch in save_epochs}
    training_times = {epoch: [] for epoch in save_epochs}

    os.makedirs("checkpoints", exist_ok=True)

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_data)):
        print(f"Starting fold {fold + 1}")

        x_train_full, x_val = x_data[train_idx], x_data[val_idx]
        y_train_full, y_val = y_data[train_idx], y_data[val_idx]

        x_train, x_val, y_train, y_val = train_test_split(
            x_train_full, y_train_full, test_size=0.2, random_state=42
        )

        model = create_vgg_with_trainable_dense(input_shape, n_classes)
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
        model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

        checkpoint_path = "checkpoints/lenet5_fold{fold}_epoch_{epoch:02d}.h5"

        epoch_start_time = None
        current_epoch = 0
        training_times_accumulator = 0

        def save_on_epoch(epoch, logs):
            nonlocal epoch_start_time, training_times_accumulator, current_epoch

            if epoch_start_time is not None:
                training_times_accumulator += time.time() - epoch_start_time

            epoch_start_time = time.time()
            current_epoch = epoch + 1

            if current_epoch in save_epochs:
                training_times[current_epoch].append(training_times_accumulator)
                model.save(checkpoint_path.format(fold=fold + 1, epoch=current_epoch))

        checkpoint_callback = callbacks.LambdaCallback(on_epoch_end=save_on_epoch)

        model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=max(save_epochs),
            batch_size=32,
            callbacks=[checkpoint_callback],
            verbose=1
        )

        for epoch in save_epochs:
            model_path = checkpoint_path.format(fold=fold + 1, epoch=epoch)
            with tf.keras.utils.custom_object_scope({"CALayer": CALayer}):
                saved_model = models.load_model(model_path)

            start_time = time.time()
            val_loss, val_accuracy = saved_model.evaluate(x_val, y_val, verbose=0)
            elapsed_time = time.time() - start_time

            evaluation_times[epoch].append(elapsed_time)
            metrics[epoch].append(val_accuracy)
            print(f"Fold {fold + 1}, Epoch {epoch}: Val Accuracy: {val_accuracy:.4f}, Eval Time: {elapsed_time:.2f}s")

    for epoch in save_epochs:
        print(f"Dataset: {dados}")
        mean_acc = np.mean(metrics[epoch])
        std_acc = np.std(metrics[epoch])
        mean_eval_time = np.mean(evaluation_times[epoch])
        mean_train_time = np.mean(training_times[epoch]) if training_times[epoch] else 0.0
        print(f"Epoch {epoch}: Mean Accuracy: {mean_acc:.4f}, Std Dev: {std_acc:.4f}, "
              f"Mean Eval Time: {mean_eval_time:.2f}s, Mean Train Time: {mean_train_time:.2f}s")


dados = ['clef', 'accidentals', 'mnist']
#dados = ['mnist']

for x in dados:
  cross_validate_lenet5(x)

Starting fold 1
Epoch 1/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.7251 - loss: 0.6931 - val_accuracy: 0.9156 - val_loss: 0.2125
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9643 - loss: 0.1268 - val_accuracy: 0.9438 - val_loss: 0.1585
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9863 - loss: 0.0625 - val_accuracy: 0.9438 - val_loss: 0.1522
Epoch 4/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9906 - loss: 0.0493 - val_accuracy: 0.9406 - val_loss: 0.1439
Epoch 5/100
36/40 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9991 - loss: 0.0223

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.9990 - loss: 0.0223 - val_accuracy: 0.9438 - val_loss: 0.1372
Epoch 6/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9983 - loss: 0.0182 - val_accuracy: 0.9469 - val_loss: 0.1470
Epoch 7/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0109 - val_accuracy: 0.9531 - val_loss: 0.1439
Epoch 8/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0072 - val_accuracy: 0.9531 - val_loss: 0.1478
Epoch 9/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 0.0056 - val_accuracy: 0.9500 - val_loss: 0.1546
Epoch 10/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 0.0037 - val_accuracy: 0.9531 - val_loss: 0.1538
Epoch 11/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9469 - val_loss: 0.1573
Epoch 12/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.9531 - va

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.9469 - val_loss: 0.1767
Epoch 16/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9500 - val_loss: 0.1748
Epoch 17/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.9531 - val_loss: 0.1757
Epoch 18/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9500 - val_loss: 0.1773
Epoch 19/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9531 - val_loss: 0.1751
Epoch 20/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9500 - val_loss: 0.1829
Epoch 21/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 7.3494e-04 - val_accuracy: 0.9531 - val_loss: 0.1780
Epoch 22/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 6.9005e-04 - val_accuracy:

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 1.0000 - loss: 5.1922e-04 - val_accuracy: 0.9531 - val_loss: 0.1867
Epoch 26/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 5.4851e-04 - val_accuracy: 0.9531 - val_loss: 0.1875
Epoch 27/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 4.1301e-04 - val_accuracy: 0.9531 - val_loss: 0.1881
Epoch 28/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 3.7530e-04 - val_accuracy: 0.9531 - val_loss: 0.1876
Epoch 29/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 3.7527e-04 - val_accuracy: 0.9500 - val_loss: 0.1931
Epoch 30/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 3.2833e-04 - val_accuracy: 0.9438 - val_loss: 0.1994
Epoch 31/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 4.1815e-04 - val_accuracy: 0.9531 - val_loss: 0.1913
Epoch 32/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 3.

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 1.0000 - loss: 1.2780e-04 - val_accuracy: 0.9500 - val_loss: 0.2178
Epoch 51/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 9.5638e-05 - val_accuracy: 0.9500 - val_loss: 0.2151
Epoch 52/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 1.4890e-04 - val_accuracy: 0.9500 - val_loss: 0.2185
Epoch 53/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 1.3041e-04 - val_accuracy: 0.9500 - val_loss: 0.2177
Epoch 54/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 9.0576e-05 - val_accuracy: 0.9500 - val_loss: 0.2181
Epoch 55/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 8.4872e-05 - val_accuracy: 0.9500 - val_loss: 0.2217
Epoch 56/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 1.0439e-04 - val_accuracy: 0.9531 - val_loss: 0.2172
Epoch 57/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 1.

...

Fold 4, Epoch 5: Val Accuracy: 0.9366, Eval Time: 3.20s


Fold 4, Epoch 15: Val Accuracy: 0.9446, Eval Time: 3.18s


Fold 4, Epoch 25: Val Accuracy: 0.9473, Eval Time: 3.09s


Fold 4, Epoch 50: Val Accuracy: 0.9485, Eval Time: 3.76s


Fold 4, Epoch 100: Val Accuracy: 0.9456, Eval Time: 3.22s
Dataset: mnist
Epoch 5: Mean Accuracy: 0.9344, Std Dev: 0.0025, Mean Eval Time: 3.17s, Mean Train Time: 52.15s
Dataset: mnist
Epoch 15: Mean Accuracy: 0.9453, Std Dev: 0.0031, Mean Eval Time: 3.17s, Mean Train Time: 183.86s
Dataset: mnist
Epoch 25: Mean Accuracy: 0.9458, Std Dev: 0.0032, Mean Eval Time: 3.09s, Mean Train Time: 315.95s
Dataset: mnist
Epoch 50: Mean Accuracy: 0.9465, Std Dev: 0.0018, Mean Eval Time: 3.25s, Mean Train Time: 643.47s
Dataset: mnist
Epoch 100: Mean Accuracy: 0.9438, Std Dev: 0.0021, Mean Eval Time: 3.14s, Mean Train Time: 1297.72s


In [ ]:
def visualize_feature_maps(model, input_image, layer_name):
    """
    Visualize feature maps from a specific layer.

    :param model: Trained Keras model.
    :param input_image: Input image as a numpy array of shape (1, height, width, channels).
    :param layer_name: Name of the layer to visualize.
    """
    print(f"Initializing model for visualization from layer: {layer_name}")

    # Check if the layer exists
    try:
        layer_output = model.get_layer(layer_name).output
    except ValueError:
        available_layers = [layer.name for layer in model.layers]
        raise ValueError(
            f"Layer '{layer_name}' not found in the model. "
            f"Available layers: {available_layers}"
        )

    # Create a submodel that outputs the desired layer's activations
    submodel = tf.keras.Model(inputs=model.input, outputs=layer_output)

    # Get feature maps
    feature_maps = submodel.predict(input_image)

    # Plot feature maps
    num_maps = feature_maps.shape[-1]
    maps_per_row = 10
    num_rows = (num_maps + maps_per_row - 1) // maps_per_row  # Calculate total rows needed

    plt.figure(figsize=(maps_per_row * 2, num_rows * 2))
    for i in range(num_maps):
        plt.subplot(num_rows, maps_per_row, i + 1)
        plt.imshow(feature_maps[0, :, :, i], cmap="gray")
        plt.axis("off")
        plt.title(f"Map {i + 1}", fontsize=8)
    plt.tight_layout()
    plt.show()


# Create the model
model = create_vgg16_with_ca(input_shape=(32, 32, 1), num_classes=3)

# Load a sample MNIST image
x_data, y_data = load_and_preprocess_clef(zip_file_path, target_size)
test_image = x_data[50]  # Shape: (28, 28, 1)

# Add batch dimension, shape becomes (1, 28, 28, 1)
test_image = np.expand_dims(test_image, axis=0)

# Initialize the model by calling it
_ = model(test_image)  # Ensures graph initialization

# Visualize feature maps
layer_name = "max_pool_4"  # Use the desired layer name
visualize_feature_maps(model, test_image, layer_name)
